In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from scipy import stats

# Загрузка данных
data_path = '/mnt/data/horse_data.csv'
data = pd.read_csv(data_path, header=None, na_values='?')  # Заменим '?' на NaN для удобства работы с пропусками

# Переименование столбцов для удобства на основе описания
data.columns = [
    "surgery", "age", "hospital_number", "rectal_temperature", "pulse", "respiratory_rate", 
    "temperature_of_extremities", "peripheral_pulse", "mucous_membranes", "capillary_refill_time", 
    "pain", "peristalsis", "abdominal_distension", "nasogastric_tube", "nasogastric_reflux", 
    "nasogastric_reflux_ph", "rectal_examination", "abdomen", "packed_cell_volume", "total_protein", 
    "abdominocentesis_appearance", "abdominocentesis_total_protein", "outcome", 
    "surgical_lesion", "type_of_lesion_1", "type_of_lesion_2", "type_of_lesion_3", "cp_data"
]

# Выбранные столбцы
selected_columns = [
    "rectal_temperature", "pulse", "respiratory_rate", "temperature_of_extremities", 
    "pain", "abdominal_distension", "packed_cell_volume", "outcome"
]
selected_data = data[selected_columns]

### Задание 1: Базовое изучение данных
# Расчет базовых статистических метрик
basic_stats = selected_data.describe(include='all').T  # Основные статистики для всех столбцов
modes = selected_data.mode().iloc[0]  # Моды для всех столбцов
basic_stats['mode'] = modes  # Добавим моду в таблицу

print("Базовые метрики для выбранных столбцов:\n", basic_stats)

# Краткое описание: можно заметить средние и медианы, распределения категорий и числовых значений

### Задание 2: Поиск выбросов
# Выбросы определяем по Z-оценке (обычно значения > 3 считаются выбросами)
numeric_columns = ["rectal_temperature", "pulse", "respiratory_rate", "packed_cell_volume"]
outliers = {}
for col in numeric_columns:
    z_scores = np.abs(stats.zscore(selected_data[col].dropna()))
    outliers[col] = selected_data[col][z_scores > 3]
    print(f"Выбросы в '{col}':\n", outliers[col])

# Гипотеза: выбросы могут быть вызваны ошибками измерений, аномалиями или экстремальными случаями.
# Решение: при наличии небольшого числа выбросов можно их удалить, иначе применить лог-трансформацию или замену на медиану.

### Задание 3: Работа с пропусками
# Количество пропусков в выбранных столбцах
missing_data = selected_data.isnull().sum()
print("\nКоличество пропусков по столбцам:\n", missing_data)

# Принятие решения по заполнению пропусков
# Например:
# - числовые: можно заполнить медианой или средним значением
# - категориальные: модой

# Заполнение пропусков
for col in selected_data.columns:
    if selected_data[col].dtype == 'float64' or selected_data[col].dtype == 'int64':
        selected_data[col].fillna(selected_data[col].median(), inplace=True)  # Числовые заполняем медианой
    else:
        selected_data[col].fillna(selected_data[col].mode()[0], inplace=True)  # Категориальные заполняем модой

print("\nДатафрейм без пропусков:\n", selected_data.head())
